In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import sys
import time
import gc
import itertools
import tqdm

from sklearn.preprocessing import LabelEncoder
import scipy.stats as ss

if os.name=='nt':
    try:
        mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin'
        os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
    except:
        pass
    
import xgboost as xgb
from sklearn.metrics import mean_squared_error 
from numba import jit

%matplotlib inline

### Load data

In [2]:
# Load data
train = pd.read_csv('all/sales_train.csv.gz')
test = pd.read_csv('all/test.csv.gz')
shop = pd.read_csv('all/shops-translated.csv')
item = pd.read_csv('all/item_category.csv')

test.set_index('ID', inplace=True)
item.drop(['item_name_translated'], axis=1, inplace=True)
shop.drop(['Name'], axis=1, inplace=True)

le = LabelEncoder()
item['item_cat1'] = le.fit_transform(item['item_cat1'].astype(str))
item['item_cat2'] = le.fit_transform(item['item_cat2'].astype(str))
shop['City'] = le.fit_transform(shop['City'])
shop['Type'] = le.fit_transform(shop['Type'])

shop.set_index('shop_id', inplace=True)
item.set_index('item_id', inplace=True)

In [7]:
train['sales'] = train['item_price']*train['item_cnt_day']
train = train.merge(shop, how='left', left_on='shop_id', right_index=True)
train = train.merge(item, how='left', left_on='item_id', right_index=True)
train.date = pd.to_datetime(train.date, format='%d.%m.%Y')

In [9]:
train.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,sales,City,Type,item_cat1,item_cat2
2935844,2015-10-10,33,25,7409,299.0,1.0,299.0,8,5,12,16
2935845,2015-10-09,33,25,7460,299.0,1.0,299.0,8,5,12,16
2935846,2015-10-14,33,25,7459,349.0,1.0,349.0,8,5,12,16
2935847,2015-10-22,33,25,7440,299.0,1.0,299.0,8,5,12,36
2935848,2015-10-03,33,25,7460,299.0,1.0,299.0,8,5,12,16


### `cnt`

In [10]:
feature_dict = {} # dict containing all the features

In [11]:
# Total count of sales per month
train_p = train.pivot_table(index=['shop_id', 'item_id'],
                           columns='date_block_num',
                           values='item_cnt_day',
                           aggfunc=np.sum).fillna(0.0)
# Rename columns
train_p.columns = ['cnt_'+str(k) for k in train_p.columns.get_level_values(0)]
# Add one column for test data
train_p['cnt_34'] = np.zeros((len(train_p), 1))
# Add to the feature dict
feature_dict['cnt'] = train_p

### `sales`

In [13]:
# Total value of sales per month
sales = train.pivot_table(index=['shop_id', 'item_id'],
                         columns='date_block_num', 
                         values='sales',
                         aggfunc=np.sum).fillna(0.0)
# Rename columns
sales.columns = ['sales_'+str(k) for k in sales.columns.get_level_values(0)]
# Add on column for test data
sales['sales_34'] = np.zeros((len(sales), 1))
# Add to the feature dict
feature_dict['sales'] = sales

### explore

In [13]:
# Apply pivot table to x with combinations of the following parameters
index_list = ['item_id', 'shop_id', 'City', 'Type', 'item_cat1', 'item_cat2']
column_list = ['date_block_num']
value_list = ['item_price', 'item_cnt_day', 'sales']
aggfunc_list = [np.sum, np.mean]

In [35]:
for idx, col, val, agf in tqdm.tqdm_notebook(list(itertools.product(index_list, column_list, value_list, aggfunc_list))):
    tmp = train.pivot_table(index=idx, columns=col, values=val, aggfunc=agf).fillna(0.0)
    tmp
    feature_dict['-'.join([idx, val, str(agf).split(' ')[1]])]

In [41]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,sales,City,Type,item_cat1,item_cat2
0,2013-01-02,0,59,22154,999.00,1.0,999.00,26.0,3.0,3.0,23.0
1,2013-01-03,0,25,2552,899.00,1.0,899.00,26.0,4.0,20.0,32.0
2,2013-01-05,0,25,2552,899.00,-1.0,-899.00,0.0,4.0,3.0,23.0
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05,1.0,5.0,3.0,23.0
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00,23.0,4.0,3.0,23.0


In [46]:
train.pivot_table(index='City', columns='item_id', values='item_cnt_day', aggfunc=np.sum)

item_id,2473,2480,2515,2522,2546,2548,2552,2554,2555,2564,...,2715,2716,2719,2720,2731,2746,2748,2798,2833,22154
City,,,,,,,,,,,,,,,,,,,,,
0.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,sales,City,Type,item_cat1,item_cat2
0,2013-01-02,0,59,22154,999.00,1.0,999.00,26.0,3.0,3.0,23.0
1,2013-01-03,0,25,2552,899.00,1.0,899.00,26.0,4.0,20.0,32.0
2,2013-01-05,0,25,2552,899.00,-1.0,-899.00,0.0,4.0,3.0,23.0
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05,1.0,5.0,3.0,23.0
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00,23.0,4.0,3.0,23.0
5,2013-01-10,0,25,2564,349.00,1.0,349.00,22.0,5.0,3.0,23.0
6,2013-01-02,0,25,2565,549.00,1.0,549.00,24.0,3.0,3.0,23.0
7,2013-01-04,0,25,2572,239.00,1.0,239.00,24.0,5.0,3.0,23.0
8,2013-01-11,0,25,2572,299.00,1.0,299.00,24.0,5.0,3.0,23.0
9,2013-01-03,0,25,2573,299.00,3.0,897.00,13.0,0.0,3.0,23.0
